<a href="https://colab.research.google.com/github/PanavShah1/DAV_assignment/blob/main/DAV_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DAV Assignment Q7
You have been provided with a dataset containing emails categorized as either spam or non-spam and another dataset with emails not yet categorized.

### Q7: Develop a model to predict whether each email is spam or not, and use it to classify the uncategorized emails.

In [ ]:
# Import libraries
import requests
from pathlib import Path
import zipfile
import os


request = requests.get("https://github.com/PanavShah1/DAV_assignment/blob/main/Email_Dataset.csv?raw=true")
with open("Email_Dataset.csv", "wb") as f:
  f.write(request.content)
  print("csv file downloaded")

csv file downloaded


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("Email_Dataset.csv")
df.head(10)

,Unnamed: 0,CATEGORY,MESSAGE
0,1000,Spam,\n\nThe Internet's Online Pharmacy\n\n\n\nViag...
1,1001,Spam,------=_NextPart_000_00B0_35C58D0E.D7267B06\n\...
2,1002,Spam,"<html>\n\n\n\n<head>\n\n<meta http-equiv=""Cont..."
3,1003,Spam,------=_NextPart_000_00E4_86E61E0A.B5488E11\n\...
4,1004,Spam,BARRISTER ADEWALE COKER CHAMBERS\n\nLegal Prac...
5,1005,Spam,<HTML>\n\n<HEAD>\n\n</HEAD>\n\n<BODY>\n\n<FONT...
6,1006,Spam,<=21DOCTYPE HTML PUBLIC =22-//W3C//DTD HTML 4....
7,1007,Spam,"<html>\n\n\n\n<head>\n\n<meta http-equiv=3D""Co..."
8,1008,Spam,Just sent you a note with the wrong link. Tthe...
9,1009,Spam,This is a multi-part message in MIME format.\n...


In [ ]:
import string
def remove_angle_brackets(text):
  return pd.Series(text).str.replace(r'<(.|\n)*>', '', regex=True)

def remove_new_line(text):
  return pd.Series(text).str.replace(r'\n', '', regex=True)

punctuation_list = string.punctuation
def remove_punctuations(text):
    temp = str.maketrans('', '', punctuation_list)
    return text.translate(temp)

df['MESSAGE'] = df['MESSAGE'].apply(remove_angle_brackets)
df['MESSAGE'] = df['MESSAGE'].apply(remove_new_line)
df['MESSAGE'] = df['MESSAGE'].apply(remove_punctuations)
df

,Unnamed: 0,CATEGORY,MESSAGE
0,1000,Spam,The Internets Online PharmacyViagra Xenical ...
1,1001,Spam,NextPart00000B035C58D0ED7267B06ContentType tex...
2,1002,Spam,httpxentcommailmanlistinfofork
3,1003,Spam,NextPart00000E486E61E0AB5488E11ContentType tex...
4,1004,Spam,BARRISTER ADEWALE COKER CHAMBERSLegal Practiti...
...,...,...,...
3995,4995,Not Spam,BEGIN PGP SIGNED MESSAGEHash SHA1A bull market...
3996,4996,Not Spam,On Thursday 25 July 2002 0616 am Kylus wrote M...
3997,4997,Not Spam,Update of cvsrootspamassassinspamassassinwordf...
3998,4998,Not Spam,On Thu 20020815 at 1053 Erik Williamson wrote ...


In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,CATEGORY,MESSAGE
0,Spam,The Internets Online PharmacyViagra Xenical ...
1,Spam,NextPart00000B035C58D0ED7267B06ContentType tex...
2,Spam,httpxentcommailmanlistinfofork
3,Spam,NextPart00000E486E61E0AB5488E11ContentType tex...
4,Spam,BARRISTER ADEWALE COKER CHAMBERSLegal Practiti...
...,...,...
3995,Not Spam,BEGIN PGP SIGNED MESSAGEHash SHA1A bull market...
3996,Not Spam,On Thursday 25 July 2002 0616 am Kylus wrote M...
3997,Not Spam,Update of cvsrootspamassassinspamassassinwordf...
3998,Not Spam,On Thu 20020815 at 1053 Erik Williamson wrote ...


In [ ]:
df.replace('Spam', value=1, inplace=True)
df.replace('Not Spam', value=0, inplace=True)
df

,CATEGORY,MESSAGE
0,1,The Internets Online PharmacyViagra Xenical ...
1,1,NextPart00000B035C58D0ED7267B06ContentType tex...
2,1,httpxentcommailmanlistinfofork
3,1,NextPart00000E486E61E0AB5488E11ContentType tex...
4,1,BARRISTER ADEWALE COKER CHAMBERSLegal Practiti...
...,...,...
3995,0,BEGIN PGP SIGNED MESSAGEHash SHA1A bull market...
3996,0,On Thursday 25 July 2002 0616 am Kylus wrote M...
3997,0,Update of cvsrootspamassassinspamassassinwordf...
3998,0,On Thu 20020815 at 1053 Erik Williamson wrote ...


In [ ]:
df.shape

(4000, 2)

In [ ]:
X = df['MESSAGE']
y = df['CATEGORY']
X.shape, y.shape

((4000,), (4000,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3200,), (800,), (3200,), (800,))

In [ ]:
X_train[:5]

3994    Padraig Brady wrote Vincent Cunniffe wrote  Po...
423                                                      
2991    URL httpboingboingnet85534710Date Not supplied...
1221    Once upon a time Angles wrote  Matthias Saou m...
506     GreetingsYou are receiving this letter because...
Name: MESSAGE, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
X_train_vectorized.shape, X_test_vectorized.shape

((3200, 59378), (800, 59378))

In [ ]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [ ]:
# Convert to dense matrix (be careful with large matrices)
dense_matrix = X_train_vectorized.toarray()
print(dense_matrix)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
pd.DataFrame(dense_matrix)

,0,1,2,3,4,5,6,7,8,9,...,59368,59369,59370,59371,59372,59373,59374,59375,59376,59377
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(X_train_vectorized)

  (0, 32397)	0.06336088891255288
  (0, 33342)	0.0631623883486335
  (0, 28587)	0.06336088891255288
  (0, 55009)	0.06336088891255288
  (0, 27923)	0.06445740114619083
  (0, 23611)	0.05308571441184109
  (0, 55403)	0.05817878376516117
  (0, 32296)	0.05843134790326897
  (0, 29555)	0.06303123691007342
  (0, 54773)	0.1538308015128526
  (0, 29706)	0.1538308015128526
  (0, 41732)	0.06647057304983893
  (0, 42324)	0.1538308015128526
  (0, 46512)	0.10127757813768387
  (0, 51019)	0.10898656637735168
  (0, 52682)	0.1538308015128526
  (0, 42323)	0.14110384064349324
  (0, 46523)	0.1370066743771144
  (0, 36153)	0.05958585587944588
  (0, 21023)	0.09436502702057838
  (0, 13127)	0.072585646780974
  (0, 19087)	0.13082866614278146
  (0, 48122)	0.08991132288490954
  (0, 52911)	0.051519794104179624
  (0, 32668)	0.06955220713176138
  :	:
  (3199, 6100)	0.22497772827366924
  (3199, 40862)	0.22497772827366924
  (3199, 38854)	0.1877513377711518
  (3199, 56447)	0.17272388296982885
  (3199, 24364)	0.2851815963212686

In [ ]:
X_train_vectorized.shape

(3200, 59378)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [ ]:
model.fit(X_train_vectorized, y_train)

LogisticRegression()

In [ ]:
train_predictions = model.predict(X_train_vectorized)
train_predictions[:10]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [ ]:
train_accuracy = model.score(X_train_vectorized, y_train)
train_accuracy

0.9134375

In [ ]:
test_predictions = model.predict(X_test_vectorized)
test_predictions[:10]

array([1, 0, 1, 0, 0, 0, 0, 1, 0, 1])

In [ ]:
test_accuracy = model.score(X_test_vectorized, y_test)
test_accuracy

0.8825

In [ ]:
X_test

555     This is a multipart message in MIME formatNext...
3491    On Mon 19 Aug 2002 164430 0200 Matthias Saou h...
527     DEAR SIRI am MRS MARIAM ABACHA wife of the lat...
3925    Tom wrote its a word that deserves a place in ...
2989     In forteanay That Goddess Chick Its official ...
                              ...                        
1922    On Aug 23 Bort Paul wroteIf your sendmail has ...
865     Dear fork 2C3CBODY bgColor3Dffccff3E3CTABLE bo...
3943    On Tue Aug 20 2002 at 054754PM 0100 Matt Berns...
1642    Im not sure if this is really a razor problemi...
2483    use Perl Daily NewsletterIn this issue     Per...
Name: MESSAGE, Length: 800, dtype: object

In [ ]:
model.predict(X_test_vectorized[0])

array([1])

## Predict

In [ ]:
request = requests.get("https://github.com/PanavShah1/DAV_assignment/blob/main/Email%20Test%20Data.csv?raw=true")
with open("Email_Test_Data.csv", "wb") as f:
  f.write(request.content)
  print("csv file downloaded")

csv file downloaded


In [ ]:
df1 = pd.read_csv("Email_Test_Data.csv")
df1

,Unnamed: 0,MESSAGE
0,0,"Dear Homeowner,\n\n \n\nInterest Rates are at ..."
1,1,ATTENTION: This is a MUST for ALL Computer Use...
2,2,This is a multi-part message in MIME format.\n...
3,3,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...
4,4,This is the bottom line. If you can GIVE AWAY...
...,...,...
995,995,"<HTML><BODY BGCOLOR=3D""#FFFFFF"">\n\n<table wid..."
996,996,Long time no chat!\n\n\n\nHow have you been? I...
997,997,"\n\n\n\nChina's rapid economic growth, as rank..."
998,998,"<html>\n\n\n\n<body>\n\n\n\n<font size=""2"" PTS..."


In [ ]:
from copy import deepcopy
df2 = deepcopy(df1)
df2

,Unnamed: 0,MESSAGE
0,0,"Dear Homeowner,\n\n \n\nInterest Rates are at ..."
1,1,ATTENTION: This is a MUST for ALL Computer Use...
2,2,This is a multi-part message in MIME format.\n...
3,3,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...
4,4,This is the bottom line. If you can GIVE AWAY...
...,...,...
995,995,"<HTML><BODY BGCOLOR=3D""#FFFFFF"">\n\n<table wid..."
996,996,Long time no chat!\n\n\n\nHow have you been? I...
997,997,"\n\n\n\nChina's rapid economic growth, as rank..."
998,998,"<html>\n\n\n\n<body>\n\n\n\n<font size=""2"" PTS..."


In [ ]:
df1['MESSAGE'] = df1['MESSAGE'].apply(remove_angle_brackets)
df1['MESSAGE'] = df1['MESSAGE'].apply(remove_new_line)
df1['MESSAGE'] = df1['MESSAGE'].apply(remove_punctuations)
df1

,Unnamed: 0,MESSAGE
0,0,Dear Homeowner Interest Rates are at their low...
1,1,ATTENTION This is a MUST for ALL Computer User...
2,2,This is a multipart message in MIME formatNext...
3,3,IMPORTANT INFORMATIONThe new domain names are ...
4,4,This is the bottom line If you can GIVE AWAY ...
...,...,...
995,995,
996,996,Long time no chatHow have you been If youve be...
997,997,Chinas rapid economic growth as ranked by the ...
998,998,


In [ ]:
X1 = df1['MESSAGE']
X1.shape

(1000,)

In [ ]:
X1_vectorized = vectorizer.transform(X1)
print(X1_vectorized)

  (0, 55002)	0.13045758069565397
  (0, 53265)	0.2678363758798638
  (0, 47510)	0.1754348357914126
  (0, 47340)	0.14182897616793208
  (0, 46615)	0.13341765532294003
  (0, 46148)	0.1437476762509228
  (0, 43624)	0.21638467546126483
  (0, 42998)	0.15154733366758943
  (0, 42990)	0.1591043847953865
  (0, 42646)	0.16975459262663029
  (0, 40846)	0.12398313537281369
  (0, 40650)	0.1763354232020991
  (0, 39169)	0.18975762117225403
  (0, 37374)	0.18346904113408694
  (0, 36178)	0.1444137157356016
  (0, 32850)	0.19422563421795927
  (0, 32542)	0.20135925214994713
  (0, 31782)	0.2678363758798638
  (0, 30690)	0.07764349084006117
  (0, 28102)	0.21055608521398542
  (0, 27433)	0.16429991002085137
  (0, 25149)	0.2548741677005965
  (0, 25111)	0.12328049744233742
  (0, 24554)	0.1206148925560548
  (0, 22408)	0.135132864912527
  :	:
  (999, 11929)	0.0730994893293776
  (999, 11707)	0.08418055150609013
  (999, 11704)	0.10300626013160948
  (999, 11100)	0.06175015286351607
  (999, 11086)	0.12601826627829502
  (999

In [ ]:
spam_predictions_binary = model.predict(X1_vectorized)
spam_predictions_binary

array([1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,

In [ ]:
spam_predictions = [0 for i in range(len(spam_predictions_binary))]

for i, x in enumerate(spam_predictions_binary):
  if x == 1:
    spam_predictions[i] = 'Spam'
  else:
    spam_predictions[i] = 'Not Spam'

spam_predictions[:10]

['Spam',
 'Not Spam',
 'Spam',
 'Spam',
 'Not Spam',
 'Spam',
 'Not Spam',
 'Not Spam',
 'Not Spam',
 'Not Spam']

In [ ]:
df2['CATEGORY'] = spam_predictions
df2

,Unnamed: 0,MESSAGE,CATEGORY
0,0,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",Spam
1,1,ATTENTION: This is a MUST for ALL Computer Use...,Not Spam
2,2,This is a multi-part message in MIME format.\n...,Spam
3,3,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,Spam
4,4,This is the bottom line. If you can GIVE AWAY...,Not Spam
...,...,...,...
995,995,"<HTML><BODY BGCOLOR=3D""#FFFFFF"">\n\n<table wid...",Not Spam
996,996,Long time no chat!\n\n\n\nHow have you been? I...,Not Spam
997,997,"\n\n\n\nChina's rapid economic growth, as rank...",Not Spam
998,998,"<html>\n\n\n\n<body>\n\n\n\n<font size=""2"" PTS...",Not Spam


In [ ]:
with open('Edited_Email_Test_Data.csv', 'wb') as f:
  f.write(df2.to_csv(index=False).encode('utf-8'))
  print("csv file created")

csv file created
